## Safety and Privacy with LangChain

In this notebook we will use our privacy (PII entity recognizer and anonymizer) and toxic text classifier `tensor-trek/distilbert-toxicity-classifier` along with LangChain to implement checks of the text going into our LLM and text generated by the LLM. For this example we will use HuggingFace Hub LLM with LangChain and a custom `PrivacyAndSafetyChain` chain that implements the two checks.

Note: You will need a HuggingFace Hub API Key for this notebook. Get your API Key from Huggingface hub - https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

Let's install some dependencies first
- You will need `transformers`, PyTorch, `langchain`, `presidio-analyzer`, `presidio-anonymizer`, and `spacy` libraries
- You will also need the spacy `en_core_web_lg` model. You can also work with `en_core_web_md` model here.

In [ ]:
!pip3 install transformers
!pip3 install torch torchvision
!pip3 install langchain presidio-analyzer presidio-anonymizer spacy huggingface-hub
!python -m spacy download en_core_web_lg

## Import the `PrivacyAndSafetyChain` custom chain

The directory `PrivacyAndSafety` contains files that implements the custom Chain.
- The `privacy_and_safety.py` file contains a Subclass of the base LangChain `Chain` class
- The `check.py` file contains the actual toxic text classification and PII entity detection and anonymization

Let's import and initialize `PrivacyAndSafetyChain` first.

In [3]:
from PrivacyAndSafety import PrivacyAndSafetyChain

safety_privacy = PrivacyAndSafetyChain(verbose=True)

We have now initialized `PrivacyAndSafetyChain` with default options. We will test it out with a HuggingFace Hub hosted `google/flan-t5-xl` LLM with HuggingFace hub.

In [1]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "<YOUR HF TOKEN HERE>"
repo_id = "google/flan-t5-xxl"

In [5]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

template = """{question}"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
)

chain = (
    prompt 
    | safety_privacy 
    | {"input": (lambda x: x['output'] ) | llm}
    | safety_privacy 
)

try:
    response = chain.invoke({"question": "What the hell is a machine learning algorithm?"})
except Exception as e:
    print(str(e))
else:
    print(response['output'])



> Entering new PrivacyAndSafetyChain chain...
Running PrivacyAndSafetyChain...
Toxic content found in text. Stopping...
Toxic content found in text. Stopping...



## Custom Configuration for `PrivacyAndSafetyChain`

We can customize the behavior of `PrivacyAndSafetyChain` via the following parameters

- `pii_mask_character`, the character used to perform anonymization of PII entities. Default is `*`
- `pii_labels` if you wish to specify a specific list of PII entity types, then a list of entity types. For a full list of PII entity labels refer [Presidio supported entities](https://microsoft.github.io/presidio/supported_entities/). Defaults to ALL entities.
- `fail_on_pii` a boolean flag which will make the chain fail if PII is detected. Defaults to `False`.
- `pii_threshold` the confidence score threshold for PII entity recognition. Defaults to 50%
- `toxicity_threshold` the confidence score threshold for toxicity classification. Defaults to 80%

In [2]:
from PrivacyAndSafety import PrivacyAndSafetyChain

safety_privacy = PrivacyAndSafetyChain(verbose=True, 
                                    pii_mask_character="#",
                                    pii_labels = ["PHONE_NUMBER", "EMAIL_ADDRESS", "PERSON", "US_SSN"],
                                    fail_on_pii = True,
                                    pii_threshold = 0.6,
                                    toxicity_threshold = 0.8)

In [3]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

template = """{question}"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
)

chain = (
    prompt 
    | safety_privacy 
    | {"input": (lambda x: x['output'] ) | llm}
    | safety_privacy 
)

try:
    response = chain.invoke({"question": """What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 338-12-6789, reminding him to store it in a safer place.
"""})
except Exception as e:
    print(str(e))
else:
    print(response['output'])

/Users/anjanbiswas/.pyenv/versions/3.11.2/envs/langchain/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)




> Entering new PrivacyAndSafetyChain chain...
Running PrivacyAndSafetyChain...
Checking for Toxic content...
Checking for PII...
PII found and fail_on_pii is True. Stopping...
PII found and fail_on_pii is True. Stopping...

